In [1]:
from google.colab import drive
drive.mount('/content/drive')
# %cd /content/drive/My Drive/Ntu/nndl/CZ4042 Final Project

%cd /content/drive/My Drive/

Mounted at /content/drive
/content/drive/My Drive


### Libraries

### Parse and clean training data

In [2]:
!pip install scikit-multilearn

     |████████████████████████████████| 89 kB 4.5 MB/s 


In [3]:
import pandas as pd
import spacy
import xml.etree.ElementTree as ET
nlp = spacy.load("en_core_web_sm")

In [4]:
restaurants_reviews_train_path = 'sameval2014/Train_Data_v2.0/Restaurants_Train_v2.xml'
restaurants_reviews_test_path = 'sameval2014/TestB/PhaseB/Restaurants_Test_Data_phaseB.xml'

In [5]:
### Get Train data

tree = ET.parse(restaurants_reviews_train_path)
root = tree.getroot()

# Use this dataframe for multilabel classification
# Must use scikitlearn's multilabel binarizer

labeled_reviews = []
for sentence in root.findall("sentence"):
    entry = {}
    aterms = []
    aspects = []
    if sentence.find("aspectTerms"):
        for aterm in sentence.find("aspectTerms").findall("aspectTerm"):
            aterms.append(aterm.get("term"))
    if sentence.find("aspectCategories"):
        for aspect in sentence.find("aspectCategories").findall("aspectCategory"):
            aspects.append(aspect.get("category"))
    entry["text"], entry["terms"], entry["aspects"]= sentence[0].text, aterms, aspects
    labeled_reviews.append(entry)
train_df = pd.DataFrame(labeled_reviews)
print("there are",len(labeled_reviews),"reviews in this training set")
#    print(sentence.find("aspectCategories").findall("aspectCategory").get("category"))

there are 3041 reviews in this training set


In [6]:
### Get Test data

tree = ET.parse(restaurants_reviews_test_path)
root = tree.getroot()

# Use this dataframe for multilabel classification
# Must use scikitlearn's multilabel binarizer

labeled_reviews = []
for sentence in root.findall("sentence"):
    entry = {}
    aterms = []
    aspects = []
    if sentence.find("aspectTerms"):
        for aterm in sentence.find("aspectTerms").findall("aspectTerm"):
            aterms.append(aterm.get("term"))
    if sentence.find("aspectCategories"):
        for aspect in sentence.find("aspectCategories").findall("aspectCategory"):
            aspects.append(aspect.get("category"))
    entry["text"], entry["terms"], entry["aspects"]= sentence[0].text, aterms, aspects
    labeled_reviews.append(entry)
test_df = pd.DataFrame(labeled_reviews)
print("there are",len(labeled_reviews),"reviews in this test set")
#    print(sentence.find("aspectCategories").findall("aspectCategory").get("category"))

there are 800 reviews in this test set


In [13]:
test_df

,text,terms,aspects
0,The bread is top notch as well.,[bread],[food]
1,I have to say they have one of the fastest delivery times in the city.,[delivery times],[service]
2,Food is always fresh and hot- ready to eat!,[Food],[food]
3,Did I mention that the coffee is OUTSTANDING?,[coffee],[food]
4,"Certainly not the best sushi in New York, however, it is always fresh, and the place is very clean, sterile.","[sushi, place]","[food, ambience]"
...,...,...,...
795,"Anyway, the owner was fake.",[owner],[service]
796,Owner is pleasant and entertaining.,[Owner],[service]
797,"I have never in my life sent back food before, but I simply had to, and the waiter argued with me over this.","[food, waiter]","[food, service]"
798,"Although the restaurant itself is nice, I prefer not to go for the food.",[food],"[ambience, food]"


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

# Convert the multi-labels into arrays
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_df.aspects)
y_test = mlb.fit_transform(test_df.aspects)
X_train = train_df.text
X_test = test_df.text

In [15]:
print(y_test)

[[0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 ...
 [0 0 1 0 1]
 [1 0 1 0 0]
 [0 0 1 0 0]]


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from skmultilearn.problem_transform import LabelPowerset
import numpy as np

# LabelPowerset allows for multi-label classification
# Build a pipeline for multinomial naive bayes classification
text_clf = Pipeline([('vect', CountVectorizer(stop_words = "english",ngram_range=(1, 1))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', LabelPowerset(MultinomialNB(alpha=1e-1))),])
text_clf = text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)

# Calculate accuracy
np.mean(predicted == y_test)

0.87675

In [17]:
# Test if SVM performs better
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', LabelPowerset(
                             SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, max_iter=6, random_state=42)))])
_ = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)

#Calculate accuracy
np.mean(predicted_svm == y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


0.86525

In [29]:
mlb.inverse_transform(y_test)

[('food',),
 ('service',),
 ('food',),
 ('food',),
 ('ambience', 'food'),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('food',),
 ('food',),
 ('food',),
 ('food',),
 ('food', 'service'),
 ('ambience', 'food'),
 ('service',),
 ('food',),
 ('food',),
 ('food', 'price'),
 ('food', 'service'),
 ('food',),
 ('service',),
 ('anecdotes/miscellaneous', 'service'),
 ('anecdotes/miscellaneous', 'food'),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('food',),
 ('food', 'service'),
 ('ambience', 'food'),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('ambience', 'food', 'service'),
 ('anecdotes/miscellaneous',),
 ('ambience', 'food'),
 ('food',),
 ('food',),
 ('service',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('food',),
 ('food',),
 ('service',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('food', 'service'),
 ('food',),
 ('food',),
 ('food', 'price'),
 ('food',),
 ('food', 'pr

In [28]:
mlb.inverse_transform(predicted)

[('food',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('food',),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('food',),
 ('food',),
 ('food',),
 ('food',),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('service',),
 ('food',),
 ('food',),
 ('food',),
 ('food', 'service'),
 ('food',),
 ('service',),
 ('service',),
 ('food',),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('food',),
 ('service',),
 ('food',),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('service',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('service',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('service',),
 ('anecdotes/miscellaneous',),
 ('food',),
 ('food',),
 ('food',),
 ('food',),
 ('food', 'price'),
 ('food',),
 ('food',),
 ('anecdotes/miscellaneous',),
 ('anecdotes/miscellaneous',),
 ('service',),
 

In [25]:
print(predicted[:20])

  (0, 2)	1
  (1, 1)	1
  (2, 2)	1
  (3, 2)	1
  (4, 2)	1
  (5, 1)	1
  (6, 2)	1
  (7, 2)	1
  (8, 2)	1
  (9, 2)	1
  (10, 2)	1
  (11, 2)	1
  (12, 1)	1
  (13, 4)	1
  (14, 2)	1
  (15, 2)	1
  (16, 2)	1
  (17, 2)	1
  (17, 4)	1
  (18, 2)	1
  (19, 4)	1


### Lets visualise our predictions

In [18]:
d = {'text': X_test, 'actual category': mlb.inverse_transform(y_test), 'pred_category': mlb.inverse_transform(predicted)}
pred_df = pd.DataFrame(data=d)

In [19]:
pd.set_option('display.max_colwidth', -1)
pred_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,text,actual category,pred_category
0,The bread is top notch as well.,"(food,)","(food,)"
1,I have to say they have one of the fastest delivery times in the city.,"(service,)","(anecdotes/miscellaneous,)"
2,Food is always fresh and hot- ready to eat!,"(food,)","(food,)"
3,Did I mention that the coffee is OUTSTANDING?,"(food,)","(food,)"
4,"Certainly not the best sushi in New York, however, it is always fresh, and the place is very clean, sterile.","(ambience, food)","(food,)"
...,...,...,...
795,"Anyway, the owner was fake.","(service,)","(service,)"
796,Owner is pleasant and entertaining.,"(service,)","(service,)"
797,"I have never in my life sent back food before, but I simply had to, and the waiter argued with me over this.","(food, service)","(food,)"
798,"Although the restaurant itself is nice, I prefer not to go for the food.","(ambience, food)","(anecdotes/miscellaneous,)"
